In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformer import EncoderDecoderTransformer
from utils import padding_collate_fn
import numpy as np

from generate_data import RandomIntegerDataset

In [105]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 64

embeddings = CustomEmbedding(vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 4.7207e-01, -1.2602e-01,  2.2463e+00,  4.5167e-01,  6.7404e-01,
          1.3216e+00,  9.1870e-01,  1.5372e+00, -7.2349e-01, -2.4614e-01,
          1.0395e+00, -1.1895e+00,  1.9467e+00,  5.0774e-02,  2.1367e+00,
         -1.4444e+00, -7.2113e-01,  1.1199e+00,  6.8713e-01, -8.7013e-01,
          7.2936e-01, -1.5331e-01, -5.1488e-01, -1.0656e-01,  3.8047e-01,
          1.4787e+00,  8.4831e-01, -9.7607e-01,  1.1987e+00, -7.2157e-01,
          1.7851e-01, -1.9990e+00,  1.8053e+00, -3.5393e-01,  3.5954e-01,
         -9.8082e-01,  2.1021e+00, -1.6648e+00, -1.6443e-02, -8.1352e-01,
         -7.4405e-01,  1.5926e+00, -6.7987e-01, -1.2246e+00, -1.1700e-01,
         -5.4412e-01, -7.7215e-03,  1.6000e+00,  8.2571e-01,  7.3580e-01,
         -1.8163e+00, -3.3404e+00,  2.1233e+00,  2.2613e+00,  7.4151e-04,
         -1.2711e+00,  4.7512e-01,  3.5768e-01, -1.3713e+00,  4.1948e-01,
         -2.9660e-01, -7.5799e-01,  8.7688e-01, -5.4629e-01],
        [ 1.9971e-01,  1.9580e-01, -2.8588e-01,  1

In [106]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 64
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

NUM_TRAINING_SEQUENCES = 10000
NUM_VALIDATION_SEQUENCES = 1000

VOCAB = [i for i in range(n_real_tokens)] # does not include SOS, EOS, PAD

VOCAB_MAP = dict()

for i, token in enumerate(VOCAB):
    VOCAB_MAP[i] = token
VOCAB_MAP[len(VOCAB_MAP)] = '<PAD>'
VOCAB_MAP[len(VOCAB_MAP) + 1] = '<SOS>'
VOCAB_MAP[len(VOCAB_MAP) + 2] = '<EOS>'

train_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_TRAINING_SEQUENCES, VOCAB)
train_dataloader = DataLoader(train_rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

val_rand_ds = RandomIntegerDataset(MIN_SEQ_LEN, MAX_SEQ_LEN, NUM_VALIDATION_SEQUENCES, VOCAB)
val_dataloader = DataLoader(val_rand_ds, batch_size = BATCH_SIZE, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [107]:
input, label = next(iter(train_dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 4,  4, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 1,  7,  2,  7,  7,  2,  3, 10, 10, 10],
        [ 6,  2, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  8,  7,  1,  3, 10, 10, 10, 10, 10],
        [ 1,  0,  3,  2,  4,  6,  3,  8,  6, 10],
        [ 2,  7,  2,  1,  8, 10, 10, 10, 10, 10],
        [ 9,  3,  2,  9,  6,  1,  3,  3, 10, 10],
        [ 8,  2,  8,  6, 10, 10, 10, 10, 10, 10],
        [ 9,  1,  4,  1,  7,  3,  2, 10, 10, 10],
        [ 3,  8,  0,  4,  7,  9,  4, 10, 10, 10],
        [ 5,  0, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 4,  0,  1,  1,  6,  5,  6,  9, 10, 10],
        [ 5,  2,  2,  2, 10, 10, 10, 10, 10, 10],
        [ 0,  5,  1,  2,  1,  5, 10, 10, 10, 10],
        [ 2,  3,  5,  3,  8,  5, 10, 10, 10, 10],
        [ 7,  7,  7,  7,  9,  0,  2,  8, 10, 10],
        [ 0,  5,  3,  5,  0,  8,  5, 10, 10, 10],
        [ 9,  6,  9,  0,  1, 10, 10, 10, 10, 10],
        [ 4,  5,  5,  3,  9,  5,  5,  4,  9,  0],
        [ 3,  0,  4, 10, 10, 10, 10, 10, 10, 10],


In [108]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 2, 
                    num_decoder_layers = 2, 
                    dim_feedforward = 32, 
                    dropout = 0.0,
                    max_context_window = MAX_CONTEXT_WINDOW,
                    use_pre_lnorm = True)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-4, momentum = 0.9, weight_decay = 1e-4)

In [109]:
def greedy_decode(source: torch.Tensor, model: nn.Module) -> torch.Tensor:
    """
    Designed to do autoregressive inference in an Encoder-Decoder transformer.

    This greedy decoder always predicts the vocabulary token corresponding to the highest logit.

    Takes the source sequence and the Encoder-Decoder to produce a predicted
    sequence starting from <SOS>.

    Note: This function *can* handle batches of sequences.

    Args:
        source - The source sequence to be passed to the Transformer's encoder block.
        model - The Encoder-Decoder transformer over which to greedy decode

    Returns:
        target - The batch of predicted sequences corresponding to the input sources.
        target_logits - (# batch elements =) batch_size (# rows =) seq_len (# cols =) vocab-dimensional vectors, each of which
        corresponds to the set of logits on a particular inference step within a given sequence.
    """
    batch_size = source.size(dim = 0)

    encoder_output, source_pad_mask = model.encode(source)

    # target will contain num_batch sequences of indices that are the predicted next-words for each batch element
    target = torch.full((batch_size, 1), SOS_TOKEN_IDX) # target.shape: [batch_size, num_loops_complete - 1]
    target_logits = torch.zeros((batch_size, 1, vocab_size))

    finished = torch.full((batch_size, ), False)

    while not finished.all() and target.size(dim = 1) <= MAX_CONTEXT_WINDOW:

        decoder_output, _ = model.decode(target, encoder_output, source_pad_mask)
        pred_logits = model.project_into_vocab(decoder_output) # pred_logits.shape: [batch_size, seq_len, vocab_size]

        last_row_pred_logits = pred_logits[:, -1, :] # last_row_pred_logits.shape == [batch_size, vocab_size]

        # Track next-word logits for loss_fn later.
        target_logits = torch.concat((target_logits, last_row_pred_logits.unsqueeze(1)), dim = 1)

        predictions = torch.argmax(last_row_pred_logits, dim = -1) # predictions.shape: [batch_size]

        # For any finished sequences (i.e. previous EOS-producers), force their prediction from this round to be a pad.
        predictions[finished] = PAD_TOKEN_IDX

        # Mark any additional sequences that just produced an EOS as finished.
        finished |= predictions == EOS_TOKEN_IDX

        target = torch.concat((target, predictions.reshape(-1, 1)), dim = 1) # target.shape: [batch_size, num_loops_complete]

    return target, target_logits[:, 1:, :]

In [150]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one training epoch (processing the entire training dataset once).
    
    Uses Teacher Forcing to train token-to-token mapping quality without cascading errors and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        optimizer - The optimizer to improve the model's weights
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.train()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    for (source, target), label in tqdm(dataloader):

        # FORWARD
        pred_logits = model(source, target)

        # pred_logits.shape: [batch_size, seq_len, vocab_size]
        # label.shape: [batch_size, seq_len]

        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

        # LOG
        with torch.no_grad():
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_gold_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, calculate_sequence_accuracy: bool = False, calculate_token_accuracy: bool = False):
    """
    Runs one validation epoch (processing the entire validation dataset once). 

    Uses Teacher Forcing (i.e. "gold") to evaluate token-to-token mapping quality and for parallelization.

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
        loss_fn - The loss function to calculate the model's correctness
        calculate_sequence_accuracy - A flag to mark whether sequence-level correctness should be tracked
        calculate_token_accuracy - A flag to mark whether token-level correctness should be tracked
    """
    model.eval()

    num_sequences = len(dataloader.dataset)
    num_tokens = 0

    epoch_loss = 0.0
    total_correct_sequences = 0
    total_correct_tokens = 0

    with torch.no_grad():
        
        for (source, target), label in tqdm(dataloader):
            
            # FORWARD
            pred_logits = model(source, target)
            batch_loss = loss_fn(pred_logits.view(-1, pred_logits.size(-1)), label.view(-1))

            # LOG
            epoch_loss += batch_loss.item()

            predictions = torch.argmax(pred_logits, dim = -1) # predictions.shape: [batch_size, seq_len]
            match_matrix = torch.eq(predictions, label)

            if calculate_sequence_accuracy:
                num_correct_sequences = torch.all(match_matrix, dim = 1).sum()
                total_correct_sequences += num_correct_sequences.item()

            if calculate_token_accuracy:
                num_correct_tokens = match_matrix.sum()      
                total_correct_tokens += num_correct_tokens.item()

                num_tokens += torch.numel(label)

    average_epoch_loss = epoch_loss / num_sequences
    average_epoch_sequence_accuracy = total_correct_sequences / num_sequences if calculate_sequence_accuracy else None
    average_epoch_token_accuracy = total_correct_tokens / num_tokens if calculate_token_accuracy else None

    return average_epoch_loss, average_epoch_sequence_accuracy, average_epoch_token_accuracy

def run_autoregressive_validation_loop(dataloader: DataLoader, model: nn.Module):
    """
    Runs one autoregressive validation epoch (processing the entire validation dataset once). 

    Args:
        dataloader - The dataloader to process the dataset in BATCH_SIZE batches
        model - The Encoder-Decoder that is being trained
    """
    model.eval()

    correct_sequences = 0
    total_sequences = 0

    with torch.no_grad():
        
        for (source, _), label in tqdm(dataloader):

            # FORWARD
            pred_indices, pred_logits = greedy_decode(source, model)

            np_source_indices = source.numpy().copy()
            np_pred_target_indices = pred_indices.numpy().copy()

            token_values = np.array(list(VOCAB_MAP.values()))
            predicted_source_tokens = token_values[np_source_indices]
            predicted_target_tokens = token_values[np_pred_target_indices]

            for s, t in zip(predicted_source_tokens, predicted_target_tokens):
                source_end_index = np.argmax(s == '<PAD>')  if '<PAD>' in s else len(s)
                target_end_index = np.argmax(t == '<EOS>')
                if np.array_equal(np.sort(s[:source_end_index]), t[1:target_end_index]):
                    correct_sequences += 1
                else:
                    print('INCORRECT SEQUENCE:')
                    print(np.sort(s[:source_end_index]))
                    print(t[1:target_end_index])
                
                print(f'{'Source:':<20} {s}\n{'Predicted Target:':<20} {t}', end = '\n\n')

            total_sequences += predicted_target_tokens.shape[0]

    return correct_sequences / total_sequences


In [111]:
EPOCHS = 10

training_losses = list()
training_sequence_accuracies = list()
training_token_accuracies = list()

gold_validation_losses = list()
gold_validation_sequence_accuracies = list()
gold_validation_token_accuracies = list()

for i in range(EPOCHS):
    # print(f'Running epoch {i+1}...')

    training_loss, training_sequence_accuracy, training_token_accuracy = run_train_epoch(train_dataloader, model, loss_fn, optim, calculate_sequence_accuracy = True, calculate_token_accuracy = True)

    training_losses.append(training_loss)
    training_sequence_accuracies.append(training_sequence_accuracy)
    training_token_accuracies.append(training_token_accuracy)

    gold_val_loss, gold_val_sequence_accuracy, gold_val_token_accuracy = run_gold_validation_loop(val_dataloader, model, loss_fn, calculate_sequence_accuracy = True, calculate_token_accuracy = True)
    
    gold_validation_losses.append(gold_val_loss)
    gold_validation_sequence_accuracies.append(gold_val_sequence_accuracy)
    gold_validation_token_accuracies.append(gold_val_token_accuracy)

print(training_losses)
print(training_sequence_accuracies)
print(training_token_accuracies)

print()

print(gold_validation_losses)
print(gold_validation_sequence_accuracies)
print(gold_validation_token_accuracies)

100%|██████████| 16/16 [00:00<00:00, 88.45it/s]

[3.5804623285293578, 0.5518355696201325, 0.22216399356126784, 0.4915331672489643, 0.0925589654147625, 0.10150856851935387, 0.014382730996608735, 0.005941895550861955, 0.0037636492390185593, 0.0029660229267552494]
[0.0169, 0.0669, 0.0924, 0.083, 0.1019, 0.1017, 0.1066, 0.1074, 0.1074, 0.1076]
[0.5183, 0.6194181818181819, 0.629582484725051, 0.6217363636363636, 0.633464867617108, 0.6332423580786026, 0.6348090909090909, 0.6349545454545454, 0.6349636363636364, 0.6351484068092536]

[0.879590202331543, 0.4458557834625244, 0.06612485456466675, 0.08747343063354492, 0.08366290640830994, 0.016694255769252778, 0.010147660106420518, 0.006259733468294144, 0.0050109468549489975, 0.004245980329811573]
[0.042, 0.095, 0.107, 0.106, 0.105, 0.109, 0.108, 0.109, 0.109, 0.109]
[0.601, 0.6153636363636363, 0.6254545454545455, 0.6261818181818182, 0.6247272727272727, 0.6268181818181818, 0.6267272727272727, 0.6266363636363637, 0.6268181818181818, 0.6268181818181818]


In [151]:
acc = run_autoregressive_validation_loop(val_dataloader, model)
print(acc)

 25%|██▌       | 4/16 [00:00<00:00, 15.21it/s]

Source:              ['0' '5' '7' '6' '6' '5' '8' '1' '4' '<PAD>']
Predicted Target:    ['<SOS>' '0' '1' '4' '5' '5' '6' '6' '7' '8' '<EOS>' '<PAD>']

Source:              ['8' '5' '2' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '5' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>']

Source:              ['6' '8' '0' '7' '8' '7' '2' '4' '8' '<PAD>']
Predicted Target:    ['<SOS>' '0' '2' '4' '6' '7' '7' '8' '8' '8' '<EOS>' '<PAD>']

Source:              ['8' '9' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '8' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>']

Source:              ['9' '9' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '9' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>']

Source:              ['7' '5' '3' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' 

 50%|█████     | 8/16 [00:00<00:00, 15.56it/s]

Source:              ['0' '8' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>']

Source:              ['3' '2' '5' '5' '9' '1' '9' '6' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '2' '3' '5' '5' '6' '9' '9' '<EOS>' '<PAD>' '<PAD>']

Source:              ['8' '1' '7' '7' '3' '7' '2' '8' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '2' '3' '7' '7' '7' '8' '8' '<EOS>' '<PAD>' '<PAD>']

Source:              ['8' '1' '2' '6' '7' '5' '6' '9' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '2' '5' '6' '6' '7' '8' '9' '<EOS>' '<PAD>' '<PAD>']

Source:              ['4' '7' '9' '8' '8' '8' '2' '8' '7' '<PAD>']
Predicted Target:    ['<SOS>' '2' '4' '7' '7' '8' '8' '8' '8' '9' '<EOS>' '<PAD>']

Source:              ['8' '4' '5' '2' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '4' '5' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>'

 62%|██████▎   | 10/16 [00:00<00:00, 15.47it/s]

Source:              ['7' '6' '0' '4' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '4' '6' '7' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['3' '6' '6' '8' '8' '0' '9' '7' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '3' '6' '6' '7' '8' '8' '9' '<EOS>' '<PAD>' '<PAD>']

Source:              ['5' '8' '8' '5' '2' '5' '4' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '2' '4' '5' '5' '5' '8' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>']

Source:              ['0' '2' '6' '7' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '2' '6' '7' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['0' '0' '9' '6' '7' '5' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '0' '5' '6' '7' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Source:              ['4' '6' '5' '3' '4' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '3

 88%|████████▊ | 14/16 [00:00<00:00, 13.62it/s]

Source:              ['5' '4' '1' '0' '1' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '1' '1' '4' '5' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['4' '8' '5' '3' '8' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '3' '4' '5' '8' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['4' '1' '9' '2' '8' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '2' '4' '8' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['6' '7' '3' '5' '0' '4' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '3' '4' '5' '6' '7' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']

Source:              ['8' '5' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '5' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>' '<PAD>']

Source:              ['6' '2' '4' '2' '6' '0' '1' '3' '1' '<PAD>'

100%|██████████| 16/16 [00:01<00:00, 14.68it/s]

Source:              ['8' '0' '4' '4' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '4' '4' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['4' '2' '0' '0' '6' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '0' '2' '4' '6' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['8' '6' '8' '6' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '6' '6' '8' '8' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['6' '7' '9' '1' '5' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '1' '5' '6' '7' '9' '<EOS>' '<PAD>' '<PAD>' '<PAD>' '<PAD>'
 '<PAD>']

Source:              ['5' '2' '6' '3' '0' '6' '5' '0' '<PAD>' '<PAD>']
Predicted Target:    ['<SOS>' '0' '0' '2' '3' '5' '5' '6' '6' '<EOS>' '<PAD>' '<PAD>']

Source:              ['7' '3' '9' '7' '1' '<PAD>' '<PAD>' '<PAD>' '<PAD>' '<PAD>']
Predic